## Set Up

#### Libraries

In [ ]:
import sys
sys.path.append("self_check_gpt")
import os
import re
from tqdm import tqdm
from dotenv import load_dotenv
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from baseline.helper.utils import read_data
from self_check_gpt import modeling_selfcheck

#### Datasets

In [2]:
dataset_halu_eval, dataset_self_check_gpt = read_data()

Load HaluEval 2.0
Length of Bio-Medical: 200.
Length of Education: 200.
Length of Finance: 200.
Length of Open-Domain: 200.
Length of Science: 200.

Loading SelfCheckGPT
The length of the dataset: 238.


#### Metrics

In [3]:
load_dotenv()
api_key = os.getenv('UPSTAGE_API_KEY')
self_check_gpt_prompt = modeling_selfcheck.SelfCheckAPIPrompt(
    client_type="openai", 
    base_url="https://api.upstage.ai/v1/solar", 
    model="solar-pro", 
    api_key=api_key)
self_check_gpt_ngram = modeling_selfcheck.SelfCheckNgram(
    n=1
)

Initiate OpenAI client... model = solar-pro
SelfCheck-1gram initialized


## Testing
### Benchmark: Halu Eval

In [4]:
samples = pd.Series(dataset_halu_eval['Bio-Medical'])[:3]

In [5]:
samples[0]

{'id': 10,
 'user_query': 'Name synonym of Acrokeratosis paraneoplastica.',
 'chatgpt_response': 'The possible synonyms for Acrokeratosis paraneoplastica are:\n- Bazex syndrome\n- Bazex-Dupré-Christol syndrome\n- Acrokeratosis neoplastica\n- Triad syndrome\n- Bazex syndrome of multiple basal cell carcinomas\n- Bazex syndrome of carcinomas of pharynx and esophagus\n- Fingerprint-acral-keratoderma - Specifically for the clinical presentation of palmoplantar keratoderma with dermatoscopic abnormal hand fingerprint patterns.',
 'chatgpt_fact': ['Bazex syndrome is a synonym for Acrokeratosis paraneoplastica.',
  'Bazex-Dupré-Christol syndrome is another name for Acrokeratosis paraneoplastica.',
  'Acrokeratosis neoplastica and Triad syndrome are also synonymous with Acrokeratosis paraneoplastica.',
  "'Bazex syndrome of multiple basal cell carcinomas' and 'Bazex syndrome of carcinomas of pharynx and esophagus' are alternative names for Acrokeratosis paraneoplastica.",
  "The term 'Fingerpri

#### SelfCheckGPT Unigram

In [ ]:
scores_selfcheckgpt_unigram_halueval = {}

for i in tqdm(range(len(samples))):
    sample = samples[i]
    idx = sample['id']
    prompt = sample['user_query']
    response = sample['chatgpt_response']
    
    setences = re.split(r'\.|\n', response)
    sentences = [s.strip() for s in setences if s.strip()]
    sampled_passages = self_check_gpt_ngram.get_sample_passages(prompt)
    
    scores_prompt[sample['id']] = selfcheck_prompt.predict(
        sentences=sentences,
        sampled_passages=sampled_passages,
        verbose=True
    )
scores_prompt = 

#### SelfCheckGPT Prompt API - Solar Pro

In [ ]:
scores_selfcheckgpt_prompt_halueval = {}

for i in tqdm(range(len(samples))):
    sample = samples[i]
    prompt = sample['user_query']
    response = sample['chatgpt_response']
    setences = re.split(r'\.|\n', response)
    sentences = [s.strip() for s in setences if s.strip()]
    sampled_passages = selfcheck_prompt.get_sample_passages(prompt)
    
    scores_prompt[sample['id']] = selfcheck_prompt.predict(
        sentences=sentences,
        sampled_passages=sampled_passages,
        verbose=True
    )
scores_prompt = 

### Benchmark: SelfCheckGPT

#### SelfCheckGPT